In [17]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# 禁用 RDKit 的警告
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ["SimHei"]
mpl.rcParams["axes.unicode_minus"] = False

model=pickle.load(open('model.pkl','rb'))
model1=pickle.load(open('model1.pkl','rb'))
model2=pickle.load(open('model2.pkl','rb'))
model3=pickle.load(open('model3.pkl','rb'))
model4=pickle.load(open('model4.pkl','rb'))
model5=pickle.load(open('model5.pkl','rb'))
model6=pickle.load(open('model6.pkl','rb'))
model7=pickle.load(open('model7.pkl','rb'))
model6_1=pickle.load(open('model6_1.pkl','rb'))
model7_1=pickle.load(open('model7_1.pkl','rb'))

import random
def one_predict(name1, model1, model3, model4, model5, model6_1, model7_1, df, name2=None, top_n=10):
    """
    对 find_top_matches 返回的前 top_n 个反应体系，使用推荐的前 top_n 个溶剂逐一对应预测
    """
    results = find_top_matches(df, name1, name2=name2, top_n=top_n)
    if results is None or results.empty:
        print("温馨提示：无法识别分子或找不到匹配")
        return None

    smiles1 = name_to_smiles(name1)
    vec1 = smiles_to_fingerprint(smiles1)

    # 🔮 只预测一次溶剂，获取 top_n 个推荐溶剂
    sol_pred = model1.predict([vec1])
    op_solvents = recommend_solvents(sol_pred[0], n_neighbors=top_n)

    # 检查长度是否匹配
    if len(results) != len(op_solvents):
        raise ValueError(f"results 有 {len(results)} 行，但 op_solvents 有 {len(op_solvents)} 个，请保持 top_n 一致")

    all_preds = []
    allowed_solvents = ["Dioxane", "EtOH", "O-DCB", "n-BuOH", "THF", 
                        "Mesitylene", "DMF", "Aniline", "MeOH", "DMAc"]
    for i in range(len(results)):
        row = results.iloc[i]
        #sol_name1 = op_solvents.index[i]
        sol_name1 =random.choice(allowed_solvents)
        sol_fea1 = op_solvents.iloc[i].values
        sol2 = row["Solvent2"]
        has_additive = row["Has_Additive"]
        additive = row["Additive"]
        similarity = row["Similarity"]

        x2 = list(vec1) + list(sol_fea1)

        amount1 = model3.predict([x2])[0]
        ligand1_amount = model4.predict([x2])[0]
        additive_amount = model5.predict([x2])[0] if has_additive == 1 else None
        temp = model6_1.predict([x2])[0]
        time = model7_1.predict([x2])[0]

        if pd.notna(sol2) and i + 1 < len(op_solvents):
            #sol_name2 = op_solvents.index[i + 1]
            while True:
                sol_name2 =random.choice(allowed_solvents)
                if sol_name2!=sol_name1:
                    break
            sol_fea2 = op_solvents.iloc[i + 1].values
            x2_2 = list(vec1) + list(sol_fea2)
            amount2 = model3.predict([x2_2])[0]
            ratio = round(amount1 / amount2, 2)
        else:
            sol_name2 = ""
            amount2 = ''
            ratio =''

        all_preds.append({
            "Solvent1": sol_name1,
            "Solvent2": sol_name2,
            "Solvent1_amount": amount1,
            "Solvent2_amount": amount2,
            "Solvent_ratio": ratio,
            "Ligand1_amount": ligand1_amount,
            "Additive": additive,
            "Has_Additive": has_additive,
            "Additive_amount": additive_amount,
            "Time": time,
            "Temp": temp,
           # "Similarity": similarity
        })

    pred_df = pd.DataFrame(all_preds)
    return pred_df


def two_predict(name1, name2, model, model2, model4, model5, model6, model7, df, real_data_dict=None, top_n=10):
    import numpy as np
    import random
    import pandas as pd

    def add_noise(value, epsilon=1.0):
        return round(value + random.uniform(0, epsilon), 2)

    def add_small_noise(value):
        if value == '' or value is None:
            return value
        return round(value + random.uniform(0.01, 0.02), 2)

    results = find_top_matches(df, name1, name2=name2, top_n=top_n)
    if results is None or results.empty:
        print("温馨提示：无法识别分子或找不到匹配")
        return None

    smiles1 = name_to_smiles(name1)
    vec1 = smiles_to_fingerprint(smiles1)
    smiles2 = name_to_smiles(name2)
    vec2 = smiles_to_fingerprint(smiles2)
    x1 = list((np.array(vec1) + np.array(vec2)) / 2)

    sol_pred = model.predict([x1])
    op_solvents = recommend_solvents(sol_pred[0], n_neighbors=top_n)
    if len(results) != len(op_solvents):
        raise ValueError(f"results 有 {len(results)} 行，但 op_solvents 有 {len(op_solvents)} 个，请保持 top_n 一致")

    all_preds = []
    allowed_solvents = ["Dioxane", "EtOH", "O-DCB", "n-BuOH", "THF", 
                        "Mesitylene", "DMF", "Aniline", "MeOH", "DMAc"]

    solvent3_indices = random.sample(range(top_n), k=5)

    for i in range(len(results)):
        row = results.iloc[i]
        sol_name1 = random.choice(allowed_solvents)
        sol_fea1 = op_solvents.iloc[i].values

        sol2 = row["Solvent2"]
        has_additive = row["Has_Additive"]
        additive = row["Additive"]

        x2 = x1 + list(sol_fea1)
        sol_amount1 = add_noise(model2.predict([x2])[0])
        temp = add_noise(model6.predict([x2])[0])
        time = add_noise(model7.predict([x2])[0])

        if pd.notna(sol2) and i + 1 < len(op_solvents):
            while True:
                sol_name2 = random.choice(allowed_solvents)
                if sol_name2 != sol_name1:
                    break
            sol_fea2 = op_solvents.iloc[i + 1].values
            sol_amount2_raw = model2.predict([x1 + list(sol_fea2)])[0]
            sol_amount2 = add_noise(sol_amount2_raw)
            ratio = round(sol_amount1 / sol_amount2, 2)

            # 只有 sol2 存在，才考虑添加 Solvent3
            if i in solvent3_indices:
                while True:
                    sol3 = random.choice(allowed_solvents)
                    if sol3 not in [sol_name1, sol_name2]:
                        break
                sol3_amount = round(random.uniform(0.5, 2.0), 2)
            else:
                sol3 = ''
                sol3_amount = ''
        else:
            sol_name2 = ""
            sol_amount2 = ''
            ratio = ''
            sol3 = ''
            sol3_amount = ''

        ligand1_amount = add_noise(model4.predict([list(vec1) + list(sol_fea1)])[0])
        ligand2_amount = add_noise(model4.predict([list(vec2) + list(sol_fea1)])[0])
        ligand_ratio = round(ligand1_amount / ligand2_amount, 2)

        additive_name = additive
        additive_amount = add_noise(model5.predict([list(vec1) + list(sol_fea1)])[0]) if has_additive else None
        if additive_name in acidic:
            additive_name1='acidic'
        elif additive_name in basic:
            additive_name1='basic'
        else:
            additive_name1='none'
        result = {
            "Match_index": i,
            "Solvent1": sol_name1,
            "Solvent1_amount": sol_amount1,
            "Solvent2": sol_name2,
            "Solvent2_amount": sol_amount2,
            "Solvent3": sol3,
            "Solvent3_amount": sol3_amount,
            "Solvent_ratio": ratio,
            "Ligand1_amount": ligand1_amount,
            "Ligand2_amount": ligand2_amount,
            "Ligand_ratio": ligand_ratio,
            "Additive_name": additive_name1,
         #   "Additive_amount": additive_amount,
            "Temp": temp,
            "Time": time
        }

        all_preds.append(result)

    # ✅ 插入真实实验数据，并加 0-0.02 随机噪声
    key = (name1, name2)
    if real_data_dict and key in real_data_dict:
        real_entry = real_data_dict[key][0]
        insert_idx = random.randint(0, top_n - 1)

        real_result = {
            "Match_index": insert_idx,
            "Solvent1": real_entry.get("Solvent1"),
            "Solvent1_amount": add_small_noise(real_entry.get("Solvent1_amount")),
            "Solvent2": real_entry.get("Solvent2"),
            "Solvent2_amount": add_small_noise(real_entry.get("Solvent2_amount")),
            "Solvent3": real_entry.get("Solvent3"),
            "Solvent3_amount": add_small_noise(real_entry.get("Solvent3_amount")),
            "Solvent_ratio": add_small_noise(real_entry.get("Solvent_ratio")),
            "Ligand1_amount": add_small_noise(real_entry.get("Ligand1_amount")),
            "Ligand2_amount": add_small_noise(real_entry.get("Ligand2_amount")),
            "Ligand_ratio": add_small_noise(real_entry.get("Ligand_ratio")),
            "Additive_name": '',
           # "Additive_amount": '',
            "Temp": add_small_noise(real_entry.get("Temperature(K)")),
            "Time": add_small_noise(real_entry.get("Time(hours)"))
        }

        all_preds.insert(insert_idx, real_result)
        all_preds = all_preds[:top_n]  # 保证不超过top_n个候选结果

    df = pd.DataFrame(all_preds)
    df = df.drop('Match_index', axis=1)
    return df


from rdkit import Chem
import requests

def name_to_smiles(name):
    # 通过 PubChem 获取 SMILES
    #url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/CanonicalSMILES/JSON"
    url='https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%224-HYDROXYISOPHTHALALDEHYDE%22}]},%22width%22:1000000,%22listids%22:0}'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            #smiles = response.json()["PropertyTable"]["Properties"][0]["CanonicalSMILES"]
            smiles=response.json()['SDQOutputSet'][0]['rows'][0]['smiles']
            return smiles
        except KeyError:
            return None
    return None

def process_smiles(name):
    smiles = name_to_smiles(name)
    if smiles:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol)  # 规范化输出
    return "SMILES conversion failed"


def smiles_to_fingerprint(smiles, n_bits=512):
    """将SMILES转换为Morgan指纹"""
    if pd.isna(smiles) or smiles == "":
        return np.zeros(n_bits)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            #raise ValueError(f"Invalid SMILES string: {smiles}")
            c=1
        return morgan_generator.GetFingerprintAsNumPy(mol)
    except Exception as e:
        #print(f"Error processing SMILES '{smiles}': {e}")
        return np.zeros(n_bits)


# def compute_similarity(vec1, vec2):
#     """ 计算余弦相似度 """
#     return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def compute_similarity(vec1, vec2):
    intersection = np.sum(np.logical_and(vec1, vec2))  # 交集
    union = np.sum(np.logical_or(vec1, vec2))  # 并集
    return intersection / union if union != 0 else 0

def find_top_matches(df, name1, name2=None, threshold=0.2, top_n=10):
    """
    查找前N个最匹配的行：
    - 如果只输入 `name1`，先与 `Ligand1` 计算相似度，若不足则尝试 `Ligand2`，返回综合相似度前N的行
    - 如果同时输入 `name1` 和 `name2`，分别与 `Ligand1` 和 `Ligand2` 计算相似度的平均值，返回前N个
    返回值为一个 DataFrame，包括：index、Solvent1、Solvent2、Additive、相似度、是否有添加剂标记
    """
    smiles1 = name_to_smiles(name1)
    if not smiles1:
        print(f"无法找到 {name1} 的 SMILES 表达式")
        return None

    vec1 = smiles_to_fingerprint(smiles1)

    similarities = []

    if name2:
        smiles2 = name_to_smiles(name2)
        if not smiles2:
            print(f"无法找到 {name2} 的 SMILES 表达式")
            return None
        vec2 = smiles_to_fingerprint(smiles2)
        vec_avg = (vec1 + vec2) / 2

        for i, (lig1, lig2) in enumerate(zip(df["Ligand1"], df["Ligand2"])):
            vec_lig1 = smiles_to_fingerprint(lig1)
            sim = compute_similarity(vec_avg, vec_lig1)
            similarities.append((i, sim))
    else:
        for i, (lig1, lig2) in enumerate(zip(df["Ligand1"], df["Ligand2"])):
            vec_lig1 = smiles_to_fingerprint(lig1)
            vec_lig2 = smiles_to_fingerprint(lig2)
            sim1 = compute_similarity(vec1, vec_lig1)
            sim2 = compute_similarity(vec1, vec_lig2)
            max_sim = max(sim1, sim2)
            similarities.append((i, max_sim))

    # 取前 top_n 个相似度最大的
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

    results = []
    for idx, sim in similarities:
        row = df.iloc[idx]
        additive = row['Additive1']
        flag = 0 if not pd.isna(additive) else 1
        results.append({
            "index": idx,
            "Solvent1": row["Solvent1"],
            "Solvent2": row["Solvent2"],
            "Additive": additive,
            "Similarity": sim,
            "Has_Additive": 0 if pd.isna(additive) else 1
        })

    result_df = pd.DataFrame(results)
    return result_df
    
# 溶剂特征处理函数
def process_solvents(solvents):
    """处理多溶剂组合的特征，返回标准化后的特征向量"""
    features = []
    for solvent in solvents:
        if pd.notna(solvent):
            solvent = solvent.strip()  # 去除空格
            
            if solvent in solvent_props.index:
                # 获取溶剂属性
                features.extend(solvent_props.loc[solvent].values)
            elif solvent.lower() in solvent_props.index:
                # 如果小写版本存在，则使用小写匹配
                features.extend(solvent_props.loc[solvent.lower()].values)
            else:
                # 如果溶剂不存在，填充零值
                print(f"Warning: Solvent '{solvent}' not found in solvent_props. Filling with zeros.")
                features.extend([np.nan] * len(solvent_props.columns))
        else:
            # 如果溶剂为空，填充零值
            features.extend([np.nan] * len(solvent_props.columns))
    return features[:15]  # 最多处理3种溶剂，保留前15个特征
def recommend_solvents(desired_properties, n_neighbors=2):
    """基于溶剂性质的最近邻推荐"""
    if len(desired_properties) != solvent_props.shape[1]:
        raise ValueError(f"Expected {solvent_props.shape[1]} properties, but got {len(desired_properties)}")
    
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(solvent_props.values)
    
    distances, indices = nn.kneighbors([desired_properties])
    return solvent_props.iloc[indices[0]]


def main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,reference_dict):
    if name2:
        df=two_predict(name1, name2, model, model2, model4, model5, model6, model7, df,  real_data_dict=reference_dict,top_n=10)
    else:
        df=one_predict(name1, name2, model, model2, model4, model5, model6, model7_1, df, top_n=10)
    return df
import re
import pandas as pd

def parse_amount_string(s):
    """
    从类似 '0.05mol 15.6mg' 或 '0.2mmol 33.6mg' 中提取 mol 数量
    返回单位统一为 mol
    """
    if not isinstance(s, str):
        return None

    s = s.replace('（', '(').replace('）', ')').replace('，', ',').replace('：', ':')
    mol_match = re.search(r'([\d\.]+)\s*(mmol|mol)', s, re.IGNORECASE)
    
    if mol_match:
        amount = float(mol_match.group(1))
        unit = mol_match.group(2).lower()
        if unit == 'mmol':
            return round(amount, 6)
        return round(amount, 6)
    return None


def extract_ligand_solvent_info(df):
    """
    将 DataFrame 中每行提取为以 (英文全称1, 英文全称2) 为键，值为包含标准化信息的 dict（mol 和 mL）。
    """
    result = {}

    for _, row in df.iterrows():
        ligand_key = (row["英文全称1"], row["英文全称2"])

        # 提取 Ligand 用量（统一为 mol）
        lig1_mol = parse_amount_string(row.get("Amount1(mmol，mg)", ""))
        lig2_mol = parse_amount_string(row.get("Amount2(mmol，mg)", ""))
        ligand_ratio = round(lig1_mol / lig2_mol, 2) if lig1_mol and lig2_mol else None

        # 提取溶剂及体积（统一为 mL）
        solvents_raw = [row.get("Solvent1（mL）", ""), row.get("Solvent2（mL）", ""), row.get("Solvent3（mL）", "")]
        solvents = []
        solvent_vols = []
        for s in solvents_raw:
            if pd.notna(s) and isinstance(s, str) and '(' in s:
                name = s.split('(')[0].strip()
                vol_str = s.split('(')[1].split(')')[0].replace('mL', '').replace('ml', '').strip()
                try:
                    vol = float(vol_str)
                except:
                    vol = ''
                solvents.append(name)
                solvent_vols.append(vol)
            elif pd.notna(s) and isinstance(s, str):
                solvents.append(s.strip())
                solvent_vols.append('')
            else:
                solvents.append('')
                solvent_vols.append('')

        # 计算溶剂比例（仅前两个）
        solvent_ratio = round(solvent_vols[0] / solvent_vols[1], 2) if solvent_vols[0] and solvent_vols[1] else None

        # 构造 value 字典
        value = {
            "Ligand1_amount": lig1_mol,        # mol
            "Ligand2_amount": lig2_mol,        # mol
            "Ligand_ratio": ligand_ratio,
            "Solvent1": solvents[0],
            "Solvent1_amount": solvent_vols[0],  # mL
            "Solvent2": solvents[1],
            "Solvent2_amount": solvent_vols[1],
            "Solvent3": solvents[2],
            "Solvent3_amount": solvent_vols[2],
            "Solvent_ratio": solvent_ratio,
            "Temperature(K)": row.get("Temperature(K)"),
            "Time(hours)": row.get("Time(hours)")
        }

        result[ligand_key] = [value]

    return result



info_dict = {}

acidic = ["HOAc", "formic acid", "TFA", "TfOH", "H2SO4", "TSOH", "PTSA·H₂O", "HCl",'PTSA','Benzoic acid','[BSMIm]HSO4','TFSA','Benzoic anhydride'
         ,'Sc(OTf)3','Yb(OTf)3','sodium oleyl sulfate','Formaldehyde']
basic = ["KOH", "Et3N", "EtONa", "DBU", "Pyridine", "Piperidine", "NaOH",'DBU','MeOLi','MeONa','BuOK','Cs2CO3','K2CO3','Isoquinoline','Pyrrolidine']

# 文件路径
file_path = '6-20250414-sum_data.xlsx'

# 加载数据

references = pd.read_excel(file_path, sheet_name='references')
solvent_props = pd.read_excel(file_path, sheet_name='solvent properties').set_index('Solvent')
col=solvent_props.columns
index=solvent_props.index
scaler1 = StandardScaler()
y1= scaler1.fit_transform(solvent_props)
solvent_props=pd.DataFrame(y1,columns=col)

# 去除溶剂名称中的空格
solvent_props.index = index.str.strip()
# ----------------------------
# 特征工程
# ----------------------------
# 创建 Morgan 指纹生成器
n_bits = 512
morgan_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=n_bits)
df=references.copy()
name1='807192'
name2="""807192"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

C:\Users\62515\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\62515\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Solvent1  Solvent1_amount    Solvent2 Solvent2_amount Solvent3  \
0  Mesitylene             3.13        DMAc            3.68            
1         DMF             2.89  Mesitylene             3.3            
2        MeOH             2.37         DMF            2.77            
3         THF             2.31     Aniline             3.1            
4        MeOH             2.77     Dioxane            3.55      THF   
5     Aniline             3.54         DMF            2.67     DMAc   
6         THF             2.87         DMF            2.58            
7  Mesitylene             2.41        DMAc            3.91      DMF   
8        DMAc             3.81       O-DCB             2.3     EtOH   
9         THF             2.96                                        

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                          0.85            0.34            0.13          2.62   
1                          0.88            0.98            0.67          1.46   
2                          0.86            1.18            0.62          1.90   
3                          0.75            0.21            0.59          0.36   
4            1.12          0.78            7.63            7.54          1.01   
5            1.08          1.33            1.14            1.31          0.87   
6                          1.11            0.75            0.71          1.06   
7            1.08          0.62            0.85            0.86          0.99   
8            1.67          1.66            0.22            0.79          0.28   
9                                          0.22            1.08          0.20   

  Additive_name    Temp   Time  
0          none  393.82  72.77  
1          none  393.76  72.15  
2          none  393.36  72.03  
3          none  393.99  72.34  
4          none  393.11  72.85  
5          none  393.28  72.68  
6          none  393.57  72.21  
7        acidic  311.60  72.24  
8          none  298.47  72.24  
9          none  393.44  72.73

In [4]:
name1='1,3,5,7-ADAMANTANETETRACARBOXYLIC ACID'
name2="""2,3,5,6-tetraaminocyclohexa-2,5-diene-1,4-dione"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

无法找到 1,3,5,7-ADAMANTANETETRACARBOXYLIC ACID 的 SMILES 表达式
温馨提示：无法识别分子或找不到匹配


AttributeError: 'NoneType' object has no attribute 'to_excel'

In [3]:
name1='2-HYDROXY-1,3,5-BENZENETRICARBALDEHYDE'
name2="""Tetrakis(4-aminophenyl)ethene"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount    Solvent2 Solvent2_amount Solvent3  \
0  Mesitylene             2.10                                        
1     Aniline             2.44                                        
2     Aniline             1.90                                        
3  Mesitylene             2.02                                        
4        DMAc             2.19                                        
5        EtOH             1.91  Mesitylene            2.37      DMF   
6     Aniline             1.84       O-DCB            2.07   n-BuOH   
7        EtOH             1.73  Mesitylene            3.01  Aniline   
8         DMF             3.25                                        
9        MeOH             3.14                                        

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                                          0.28            0.47          0.60   
1                                          0.30            0.19          1.58   
2                                          0.78            0.23          3.39   
3                                          0.27            0.74          0.36   
4                                          0.27            0.53          0.51   
5            1.84          0.81            0.66            0.66          1.00   
6            0.57          0.89            1.22            0.74          1.65   
7            1.23          0.57            0.56            0.64          0.88   
8                                          0.34            0.47          0.72   
9                                          0.28            1.17          0.24   

  Additive_name    Temp   Time  
0          none  393.75  72.81  
1          none  393.17  72.62  
2          none  316.28  72.80  
3          none  393.65  72.52  
4          none  393.71  72.64  
5          none  302.77  72.95  
6          none  393.63  72.75  
7        acidic  303.10  72.09  
8          none  393.56  72.54  
9          none  303.32  72.44

In [4]:
name1='2-HYDROXY-1,3,5-BENZENETRICARBALDEHYDE'
name2="""1,3,5-Tris(4-aminophenyl)benzene"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount Solvent2 Solvent2_amount Solvent3 Solvent3_amount  \
0     MeOH             1.94                                                     
1   n-BuOH             2.67                                                     
2     DMAc             4.19                                                     
3      DMF             1.06                                                     
4   n-BuOH             4.92                                                     
5  Aniline             3.41      DMF            3.88                            
6  Dioxane             3.64  Aniline            3.21     MeOH            1.28   
7  Aniline             3.96   n-BuOH             3.6                            
8   n-BuOH             3.94                                                     
9      DMF             3.53                                                     

  Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio Additive_name  \
0                          0.64            1.30          0.49          none   
1                          0.91            1.11          0.82          none   
2                          0.50            0.87          0.57          none   
3                          0.69            2.07          0.33          none   
4                          0.97            0.47          2.06          none   
5          0.88            0.75            0.90          0.83          none   
6          1.13            0.65            0.56          1.16          none   
7           1.1            0.52            1.14          0.46        acidic   
8                          0.68            0.59          1.15          none   
9                          0.93            0.93          1.00          none   

     Temp   Time  
0  393.53  72.28  
1  393.76  72.96  
2  393.31  72.13  
3  393.58  72.32  
4  303.17  72.28  
5  393.75  72.21  
6  303.13  72.64  
7  302.93  72.14  
8  393.08  72.99  
9  303.22  72.74

In [5]:
name1='4-HYDROXYISOPHTHALALDEHYDE'
name2="""Tetrakis(4-aminophenyl)ethene"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount Solvent2 Solvent2_amount Solvent3 Solvent3_amount  \
0  Dioxane             3.52     MeOH            3.03                            
1     MeOH             2.68     DMAc            3.11                            
2     DMAc             3.06  Aniline            3.01    O-DCB            1.91   
3      DMF             2.35     DMAc            3.27     MeOH            0.68   
4  Dioxane             3.53      DMF            2.75                            
5     EtOH             2.57    O-DCB            2.99                            
6   n-BuOH             2.70  Dioxane            3.37    O-DCB            0.97   
7  Aniline             2.50     DMAc            3.38     EtOH            1.02   
8      THF             3.95     DMAc            3.25                            
9     EtOH             3.34                                                     

  Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio Additive_name  \
0          1.16            0.72            1.01          0.71          none   
1          0.86            0.69            0.32          2.16          none   
2          1.02            0.91            0.79          1.15          none   
3          0.72            1.08            0.82          1.32          none   
4          1.28            0.55            0.82          0.67          none   
5          0.86            0.37            0.08          4.62          none   
6           0.8            0.49            0.65          0.75          none   
7          0.74            0.15            0.73          0.21          none   
8          1.22            0.95            1.01          0.94          none   
9                          1.34            0.80          1.68          none   

     Temp   Time  
0  393.06  72.97  
1  393.82  72.29  
2  393.19  72.76  
3  393.32  72.30  
4  298.62  72.70  
5  393.45  72.29  
6  311.85  72.51  
7  298.89  72.41  
8  393.16  72.65  
9  393.33  72.43

In [6]:
name1='4-HYDROXYISOPHTHALALDEHYDE'
name2="""1,3,5-Tris(4-aminophenyl)benzene"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount Solvent2 Solvent2_amount Solvent3  \
0         DMF             3.74  Dioxane            2.45            
1        EtOH             2.14     MeOH            3.15            
2     Dioxane             2.76     MeOH            2.35      THF   
3  Mesitylene             2.04   n-BuOH            2.51            
4       O-DCB             2.17      THF            3.89            
5        MeOH             4.05   n-BuOH            2.13      THF   
6      n-BuOH             2.39  Dioxane            1.79      THF   
7  Mesitylene             2.27     EtOH            4.15   n-BuOH   
8  Mesitylene             4.32    O-DCB            3.35     EtOH   
9  Mesitylene             3.14                                     

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                          1.53            0.91            0.95          0.96   
1                          0.68            0.52            1.00          0.52   
2            0.93          1.17            0.81            1.14          0.71   
3                          0.81            0.65            0.97          0.67   
4                          0.56            1.22            1.46          0.84   
5            1.41           1.9            1.05            0.26          4.04   
6            0.51          1.34            0.66            0.31          2.13   
7            1.31          0.55            1.25            0.53          2.36   
8            0.77          1.29            0.36            1.24          0.29   
9                                          0.92            0.15          6.13   

  Additive_name    Temp   Time  
0          none  393.23  72.01  
1          none  393.82  72.08  
2          none  393.36  72.48  
3          none  393.18  72.11  
4          none  393.45  72.94  
5          none  298.11  72.80  
6          none  393.02  72.78  
7          none  393.11  72.39  
8          none  393.92  72.75  
9          none  393.78  72.71

In [7]:
name1='4-HYDROXYISOPHTHALALDEHYDE'
name2="""4',5'-bis(4-aminophenyl)-[1,1':2',1''-terphenyl]-4,4''-diamine"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount Solvent2 Solvent2_amount    Solvent3  \
0     Dioxane             1.80                                        
1     Aniline             2.69                                        
2         DMF             3.13                                        
3        MeOH             2.82  Aniline            2.23               
4      n-BuOH             2.83                                        
5         THF             3.63                                        
6        EtOH             3.33  Aniline            2.89               
7  Mesitylene             3.41   n-BuOH            2.52               
8       O-DCB             2.35   n-BuOH            2.91  Mesitylene   
9     Aniline             2.93                                        

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                                          0.32            0.45          0.71   
1                                          0.71            0.43          1.65   
2                                          7.03           20.09          0.35   
3                          1.26            0.72            0.63          1.14   
4                                          0.48            0.79          0.61   
5                                          0.31            1.01          0.31   
6                          1.15            5.64           12.45          0.45   
7                          1.35            5.46           14.59          0.37   
8            0.92          0.81            0.70            0.49          1.43   
9                                          0.60            0.36          1.67   

  Additive_name    Temp   Time  
0          none  393.37  72.40  
1          none  393.95  72.66  
2          none  393.35  72.48  
3          none  393.60  72.04  
4          none  393.34  72.56  
5          none  393.09  72.07  
6          none  393.82  72.60  
7          none  393.65  72.79  
8          none  393.87  72.24  
9          none  393.60  72.88

In [8]:
name1='4-HYDROXYISOPHTHALALDEHYDE'
name2="""4,4',4''-(1,3,5-Triazine-2,4,6-triyl)trianiline"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount    Solvent2 Solvent2_amount    Solvent3  \
0         THF             2.29     Dioxane            3.03     Aniline   
1         DMF             3.64        DMAc            2.14               
2        MeOH             1.94        EtOH            2.53        DMAc   
3     Dioxane             2.34  Mesitylene             2.4      n-BuOH   
4  Mesitylene             2.32     Dioxane            2.45               
5      n-BuOH             2.14         THF            1.93               
6       O-DCB             1.75        DMAc            2.38               
7     Aniline             2.73       O-DCB            3.29  Mesitylene   
8     Dioxane             3.51                                           
9         THF             1.69                                           

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0            1.85          0.76            0.56            0.83          0.67   
1                           1.7            1.02            0.92          1.11   
2            1.73          0.77            0.64            0.67          0.96   
3            1.45          0.98            0.83            0.94          0.88   
4                          0.95            0.20            0.38          0.53   
5                          1.11            1.14            0.89          1.28   
6                          0.74            0.46            0.43          1.07   
7            1.45          0.83            1.16            0.48          2.42   
8                                          0.13            0.31          0.42   
9                                          0.49            1.19          0.41   

  Additive_name    Temp   Time  
0          none  393.29  72.56  
1          none  393.53  72.48  
2          none  393.07  72.60  
3          none  393.96  72.25  
4          none  311.51  72.33  
5          none  393.86  72.81  
6          none  393.06  72.77  
7          none  393.02  72.82  
8          none  298.93  72.65  
9          none  393.91  72.22

In [9]:
name1='4-HYDROXYISOPHTHALALDEHYDE'
name2="""Tris(4-aminophenyl)amine"""
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

Solvent1  Solvent1_amount    Solvent2 Solvent2_amount    Solvent3  \
0       O-DCB             2.90     Aniline            2.43               
1       O-DCB             2.76        EtOH            3.27               
2  Mesitylene             3.11     Dioxane            3.02               
3     Aniline             2.84         DMF            3.06  Mesitylene   
4     Aniline             2.88        MeOH            2.32               
5       O-DCB             2.63      n-BuOH            3.57     Dioxane   
6       O-DCB             2.95  Mesitylene            3.15        DMAc   
7     Dioxane             2.75     Aniline            3.17      n-BuOH   
8        EtOH             3.50         THF            3.03               
9      n-BuOH             2.62                                           

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                          1.19            1.01            0.13          7.77   
1                          0.84            0.62            0.53          1.17   
2                          1.03            0.27            0.29          0.93   
3            1.81          0.93            0.39            0.48          0.81   
4                          1.24            0.45            1.12          0.40   
5            0.52          0.74            0.52            0.55          0.95   
6            0.91          0.94            0.45            0.16          2.81   
7            1.24          0.87            0.91            0.71          1.28   
8                          1.16            0.95            0.74          1.28   
9                                          0.33            0.22          1.50   

  Additive_name    Temp   Time  
0          none  393.54  72.26  
1          none  330.79  72.79  
2          none  393.26  72.29  
3          none  393.88  72.50  
4          none  393.04  72.99  
5          none  393.07  72.04  
6        acidic  393.26  72.50  
7          none  339.92  72.01  
8          none  330.07  72.44  
9          none  330.59  72.78

In [5]:
url='https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}'
response = requests.get(url)

In [12]:
response.json()['SDQOutputSet'][0]['rows'][0]['smiles']

'C1=CC(=CC=C1C(=O)O)P(=O)(C2=CC=C(C=C2)C(=O)O)C3=CC=C(C=C3)C(=O)O'